# Анализ сервиса вопросов и ответов по программированию

Цель проекта - использовать навыки SQL для подсчета и визуализации ключевых метрик сервис-системы вопросов и ответов о программировании.

В проекте будет осуществляться работа с базой данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам. Мы будем работать с версией базы, где хранятся данные о постах за 2008 год. 

Проект был сделан в онлайн-тренажёре SQL, поэтому при переносе данных в Jupiter подключиться к БД и выполнять запросы активным кодом в этом учебном проекте нет возможности.

## Описание таблиц, содержащихся в базе данных

***stackoverflow.badges***

Хранит информацию о значках, которые присуждаются за разные достижения:

- id - идентификатор значка, первичный ключ таблицы;
- name - пазвание значка;
- user_id - идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице users;
- creation_date - дата присвоения значка.

***stackoverflow.post_types***

Содержит информацию о типе постов. Их может быть два: question — пост с вопросом, и answer — пост с ответом.

- id - идентификатор поста, первичный ключ таблицы;
- type - тип поста.

***posts***

Содержит информацию о постах:

- id - идентификатор поста, первичный ключ таблицы;
- title - заголовок поста;
- creation_date - дата создания поста;
- favorites_count - число, которое показывает, сколько раз пост добавили в «Закладки»;
- last_activity_date - дата последнего действия в посте, например комментария;
- last_edit_date - дата последнего изменения поста;
- user_id - идентификатор пользователя, который создал пост, внешний ключ к таблице users;
- parent_id - если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом;
- post_type_id - идентификатор типа поста, внешний ключ к таблице post_types;
- score - количество очков, которое набрал пост;
- views_count - количество просмотров.

***users***

Содержит информацию о пользователях:

- id - идентификатор пользователя, первичный ключ таблицы;
- creation_date - дата регистрации пользователя;
- display_name - имя пользователя;
- last_accsess_date - дата последнего входа;
- location - местоположение;
- reputation - очки репутации, которые получают за хорошие вопросы и полезные ответы;
- views - число просмотров профиля пользователя.

***vote_types***

Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов голосов бывает несколько:

- UpMod — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
- DownMod — такую отметку получают посты, которые показались пользователям наименее полезными.
- Close — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
- Offensive — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
- Spam — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.

Описание таблицы:

- id - идентификатор типа голоса, первичный ключ;
- name - название метки.

***votes***

Содержит информацию о голосах за посты:

- id - идентификатор голоса, первичный ключ;
- post_id - идентификатор поста, внешний ключ к таблице posts;
- user_id - идентификатор пользователя, который поставил посту голос, внешний ключ к таблице users;
- bounty_amount - сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту;
- vote_type_id - идентификатор типа голоса, внешний ключ к таблице vote_types;
- creation_date - дата назначения голоса.

1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(p.title)
FROM stackoverflow.posts p
WHERE p.score > 300 OR
p.favorites_count >= 100;

2.
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH pp AS (SELECT COUNT(p.title),
CAST(DATE_TRUNC('day', p.creation_date) AS date)
FROM stackoverflow.posts p
WHERE CAST(DATE_TRUNC('day', p.creation_date) AS date) BETWEEN '2008-11-01' AND '2008-11-18'
GROUP BY CAST(DATE_TRUNC('day', p.creation_date) AS date))
SELECT ROUND(AVG(count))
FROM pp

3.
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(DATE_TRUNC('day', u.creation_date) AS date) = CAST(DATE_TRUNC('day', b.creation_date) AS date)

4.
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT(p.id))
FROM stackoverflow.users u
INNER JOIN stackoverflow.posts p ON u.id=p.user_id
INNER JOIN stackoverflow.votes v ON v.post_id=p.id
WHERE display_name = 'Joel Coehoorn'

5.
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
RANK() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types v
GROUP BY v.name, v.id
ORDER BY v.id

6.
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT u.id,
COUNT(v.id) AS votes
FROM stackoverflow.users u
INNER JOIN stackoverflow.votes v ON u.id=v.user_id
INNER JOIN stackoverflow.vote_types vt ON vt.id=v.vote_type_id
GROUP BY u.id, vt.name
HAVING vt.name LIKE 'Close'
ORDER BY votes DESC,
u.id DESC
LIMIT 10

7.
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT u.id AS ids,
COUNT(b.id) AS badges,
DENSE_RANK() OVER (ORDER BY COUNT(b.id) DESC)
FROM stackoverflow.users u
INNER JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(DATE_TRUNC('day', b.creation_date) AS date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY ids
ORDER BY badges DESC,
ids
LIMIT 10

8.
Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT p.title,
u.id,
p.score,
ROUND(AVG(p.score) OVER (PARTITION BY p.user_id))
FROM stackoverflow.posts p
INNER JOIN stackoverflow.users u ON u.id=p.user_id
WHERE p.score <> 0 AND p.title IS NOT NULL

9.
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT p.title
FROM stackoverflow.posts p
INNER JOIN stackoverflow.users u ON p.user_id=u.id
INNER JOIN stackoverflow.badges b ON b.user_id=u.id
WHERE p.title IS NOT NULL
GROUP BY p.title
HAVING COUNT(b.id) > 1000

10.
Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT u.id,
u.views,
CASE 
WHEN u.views < 100 THEN 3
WHEN u.views >= 100 AND u.views < 350 THEN 2
WHEN u.views >= 350 THEN 1
END
FROM stackoverflow.users u
WHERE u.location LIKE '%Canada%'
AND u.views > 0

11.
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH case_v AS (SELECT id,
views,
CASE 
WHEN views < 100 THEN 3
WHEN views >= 100 AND views < 350 THEN 2
WHEN views >= 350 THEN 1
END cat
FROM stackoverflow.users 
WHERE location LIKE '%Canada%'
AND views > 0)
SELECT views AS max,
id,
cat
FROM case_v
WHERE views IN (99, 349, 21991)
ORDER BY views DESC,
id

12.
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH
t1 AS
(SELECT CAST(DATE_TRUNC('day', creation_date) AS date) AS dt, COUNT(id) AS val
FROM stackoverflow.users
GROUP BY CAST(DATE_TRUNC('day', creation_date) AS date)
ORDER BY CAST(DATE_TRUNC('day', creation_date) AS date))

SELECT RANK() OVER(ORDER BY dt) ,val,
SUM(val) OVER (ORDER BY dt) AS cum
FROM t1 
WHERE CAST(DATE_TRUNC('day', dt) AS date) BETWEEN '2008-11-01' AND '2008-11-30'

13.
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
WITH p AS 
(SELECT user_id, creation_date,
RANK() OVER (PARTITION BY user_id ORDER BY creation_date)  AS first_pub
FROM stackoverflow.posts 

ORDER BY user_id)

SELECT user_id, p.creation_date - u.creation_date FROM p
JOIN stackoverflow.users u ON p.user_id = u.id
WHERE first_pub = 1 

14.
Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT CAST(DATE_TRUNC('month', creation_date) AS date) AS month,
SUM(views_count)
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('day', creation_date) AS date) BETWEEN '2008-01-01' AND '2008-12-31'
GROUP BY CAST(DATE_TRUNC('month', creation_date) AS date)
ORDER BY sum DESC

15.
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
COUNT(DISTINCT(p.user_id))
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON u.id=p.user_id
JOIN stackoverflow.post_types pt ON pt.id=p.post_type_id
WHERE (DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month') AND (p.post_type_id=2)
GROUP BY display_name
HAVING COUNT(p.id) > 100

16.
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH
t1 AS 
(SELECT u.id
FROM stackoverflow.users AS u JOIN stackoverflow.posts AS p ON u.id=p.user_id
WHERE (u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30')
AND ((p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31'))
GROUP BY u.id)

SELECT CAST(DATE_TRUNC('month', p.creation_date) AS date) AS month, COUNT(p.id) AS cnt
FROM stackoverflow.users AS u JOIN stackoverflow.posts AS p ON u.id=p.user_id
WHERE (p.creation_date::date BETWEEN '2008-01-01' AND '2008-12-31')
AND p.user_id IN (SELECT * FROM t1)
GROUP BY CAST(DATE_TRUNC('month', p.creation_date) AS date)
ORDER BY CAST(DATE_TRUNC('month', p.creation_date) AS date) DESC

17.
Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT p.user_id,
p.creation_date,
p.views_count,
SUM(p.views_count) OVER (PARTITION BY p.user_id ORDER BY p.creation_date)
FROM stackoverflow.users u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
ORDER BY p.user_id, p.creation_date

5.
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH posts AS (SELECT user_id,
COUNT(distinct creation_date::date) AS count_p
FROM stackoverflow.posts
WHERE CAST(DATE_TRUNC('day', creation_date) AS date) BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY user_id
HAVING COUNT(id)>=1)
SELECT ROUND(AVG(count_p))
FROM posts

6.
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
with a AS (SELECT EXTRACT(month from creation_date) AS num, COUNT(id) AS cnt
           FROM stackoverflow.posts
           WHERE  creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
          GROUP BY 1)

          SELECT num, cnt, ROUND(((cnt::numeric/LAG(cnt) OVER (ORDER BY num))-1)*100,2)
          FROM a

7.
Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT
DISTINCT(EXTRACT(week FROM creation_date::date)),
MAX(creation_date) OVER (ORDER BY EXTRACT(week FROM creation_date::date))
FROM stackoverflow.posts
WHERE user_id = 22656
AND creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'